In [1]:
# ----------------------------------------------------------------------------
# Project     : mrs - multi-turn response selection
# Created By  : Eungis
# Team        : AI Engineering
# Created Date: 2023-11-30
# Updated Date: 2023-12-22
# Purpose     : Make data_loader for loading data
# version     : 0.0.1
# ---------------------------------------------------------------------------

In [2]:
import logging
import pandas as pd
from importlib import reload  # Not needed in Python 2

reload(logging)
logging.basicConfig(
    format="%(message)s",
    level=logging.DEBUG,
)

DATA_ROOT = "../data/"
logger = logging.getLogger()
data = pd.read_csv(DATA_ROOT + "smilestyle_dataset.tsv", sep="\t")

In [3]:
from typing import List

cols = data.columns.tolist()
logger.debug(f"Columns: {cols}")
# use formal conversational data
data = data[["formal"]]

data["group"] = data["formal"].isnull().cumsum()
n_sessions = data["group"].iat[-1] + 1
logger.debug(f"Number of groups: {n_sessions}")

# split data into sessions
sessions: List[List[str]] = []
groups = data.groupby("group", as_index=False, group_keys=False)

for i, group in groups:
    session = group.dropna()["formal"].tolist()
    sessions += [session]

assert n_sessions == len(sessions)

Columns: ['formal', 'informal', 'android', 'azae', 'chat', 'choding', 'emoticon', 'enfp', 'gentle', 'halbae', 'halmae', 'joongding', 'king', 'naruto', 'seonbi', 'sosim', 'translator']
Number of groups: 236


In [6]:
sessions[0]

['안녕하세요. 저는 고양이 6마리 키워요.',
 '고양이를 6마리나요? 키우는거 안 힘드세요?',
 '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
 '가장 나이가 많은 고양이가 어떻게 돼요?',
 '여섯 살입니다. 갈색 고양이에요.',
 '그럼 가장 어린 고양이가 어떻게 돼요?',
 '한 살입니다. 작년에 분양 받았어요.',
 '그럼 고양이들끼리 안 싸우나요?',
 '저희 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요.']

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
logger.debug(f"Special tokens: {tokenizer.special_tokens_map}")
logger.debug(f"EOS token & SEP token: {tokenizer.eos_token} / {tokenizer.sep_token}")
special_tokens = {"sep_token": "<SEP>"}
tokenizer.add_special_tokens(special_tokens)

Starting new HTTPS connection (1): huggingface.co:443
https://huggingface.co:443 "HEAD /klue/roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
Special tokens: {'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}
EOS token & SEP token: [SEP] / [SEP]


1

In [49]:
logger.info(
    f"""Special tokens map: {tokenizer.special_tokens_map}
{tokenizer.eos_token}: {tokenizer.eos_token_id}
{tokenizer.sep_token}: {tokenizer.sep_token_id}
{tokenizer.mask_token}: {tokenizer.mask_token_id}"""
)

Special tokens map: {'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '<SEP>', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}
[SEP]: 2
<SEP>: 32000
[MASK]: 4


In [47]:
import random

session = sessions[0]
mask_ratio = 0.15
corrupt_tokens = []
output_tokens = []
for i, utt in enumerate(session):
    original_token = tokenizer.encode(utt, add_special_tokens=False)
    n_mask = int(len(original_token) * mask_ratio)
    mask_positions = random.sample([x for x in range(len(original_token))], n_mask)
    corrupt_token = []
    for pos in range(len(original_token)):
        if pos in mask_positions:
            corrupt_token.append(tokenizer.mask_token_id)
        else:
            corrupt_token.append(original_token[pos])
    if i == len(session) - 1:
        output_tokens.extend(original_token)
        corrupt_tokens.extend(corrupt_token)
    else:
        output_tokens.extend(original_token + [tokenizer.sep_token_id])
        corrupt_tokens.extend(corrupt_token + [tokenizer.sep_token_id])

logger.debug(tokenizer.decode(output_tokens))
logger.debug(tokenizer.decode(corrupt_tokens))

안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 가장 나이가 많은 고양이가 어떻게 돼요? <SEP> 여섯 살입니다. 갈색 고양이에요. <SEP> 그럼 가장 어린 고양이가 어떻게 돼요? <SEP> 한 살입니다. 작년에 분양 받았어요. <SEP> 그럼 고양이들끼리 안 싸우나요? <SEP> 저희 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요.
안녕하세요. 저는 고양이 6마리 [MASK]요. <SEP> 고양이 [MASK] 6마리나요? 키우는 [MASK] 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 [MASK]진 않 [MASK]요. <SEP> 가장 나이가 [MASK]은 고양이가 어떻게 돼요? <SEP> 여섯 살입니다. 갈색 고양이에 [MASK]. <SEP> [MASK] 가장 어린 고양이가 어떻게 돼요? <SEP> 한 살입니다 [MASK] 작년에 분양 받았어요. <SEP> 그럼 고양이들끼리 안 싸우나 [MASK]? <SEP> 저희 일곱은 다같이 한 가족입니다 [MASK] 싸우는 일은 [MASK]어요.


In [53]:
# construct short sessions
k = 4
short_sessions = []
for session in sessions:
    for i in range(len(session) - k + 1):
        short_sessions.append(session[i : i + k])
logger.debug(len(short_sessions))
logger.info(short_sessions[0])
logger.info(short_sessions[1])

2762
['안녕하세요. 저는 고양이 6마리 키워요.', '고양이를 6마리나요? 키우는거 안 힘드세요?', '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.', '가장 나이가 많은 고양이가 어떻게 돼요?']
['고양이를 6마리나요? 키우는거 안 힘드세요?', '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.', '가장 나이가 많은 고양이가 어떻게 돼요?', '여섯 살입니다. 갈색 고양이에요.']


In [54]:
# construct negative response candidates
import random

all_utts = set()
for session in sessions:
    for utt in session:
        all_utts.add(utt)
all_utts = list(all_utts)
logger.info(f"Number of negative samples: {len(all_utts)}")

Number of negative samples: 3430


In [60]:
session = short_sessions[0]
urc_tokens = []
context_utts = []

for i in range(len(session)):
    utt = session[i]
    original_token = tokenizer.encode(utt, add_special_tokens=False)
    if i == len(session) - 1:
        positive_tokens = urc_tokens + original_token
        while True:
            random_neg_response = random.choice(all_utts)
            if random_neg_response not in context_utts:
                break
        # random negative response
        random_neg_response_token = tokenizer.encode(
            random_neg_response, add_special_tokens=False
        )
        random_tokens = urc_tokens + random_neg_response_token

        # context negative response
        context_neg_response = random.choice(context_utts)
        context_neg_response_token = tokenizer.encode(
            context_neg_response, add_special_tokens=False
        )
        context_neg_tokens = urc_tokens + context_neg_response_token
    else:
        urc_tokens += original_token + [tokenizer.sep_token_id]
    context_utts.append(utt)

logger.debug(tokenizer.decode(positive_tokens))
logger.debug(tokenizer.decode(random_tokens))
logger.debug(tokenizer.decode(context_neg_tokens))

안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 가장 나이가 많은 고양이가 어떻게 돼요?
안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 아니요, 그렇게는 절대로 살기 싫습니다.
안녕하세요. 저는 고양이 6마리 키워요. <SEP> 고양이를 6마리나요? 키우는거 안 힘드세요? <SEP> 제가 워낙 고양이를 좋아해서 크게 힘들진 않아요. <SEP> 안녕하세요. 저는 고양이 6마리 키워요.
